# Homework 5: Maps, Hypothesis Testing, and Sampling
Welcome to the last homework assignment!

Please complete this notebook by filling in the cells provided. Before you begin, execute the following cell to load the provided tests.

In [ ]:
# Don't change this cell; just run it. 
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

from client.api.notebook import Notebook
ok = Notebook('hw05.ok')
_ = ok.auth(inline=True)

Reading:
- Textbook section [7.3](https://data-8r.gitbooks.io/textbook/chapters/07/3/example-bike-sharing-in-the-bay-area.html) and chapter [8](https://data-8r.gitbooks.io/textbook/chapters/08/randomness.html)

Deadline:

This assignment is due **Tuesday, August 1 at 1PM**. You will receive an early submission bonus point if you turn in your final submission by **Monday, July 31 at 1PM**. Late work will not be accepted unless you have made special arrangements with your TA or the instructor.

Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged. 

You should start early so that you have time to get help if you're stuck. Drop-in office hours will be held at various times in the week; check the course calendar on the [course webpage](http://data8r.org) for the latest schedule.

Once you're finished, select "Save and Checkpoint" in the File menu and then execute the `submit` cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. If you submit more than once before the deadline, we will only grade your final submission.

In [ ]:
_ = ok.submit()

## 1. Mapping the Bay Area Bike Share


In lecture you were intorduced to the Bay Area Bike Share dataset and `map_table`, a powerful visualization tool. In this exercise you will get to practice your new skills.

The [Bay Area Bike Share](http://www.bayareabikeshare.com/) service published a [dataset](http://www.bayareabikeshare.com/open-data) describing every bicycle rental from September 2014 to August 2015 in their system. There were 354,152 rentals in all. The columns are:

- An ID for the rental
- Duration of the rental, in seconds
- Start date
- Name of the Start Station and code for Start Terminal
- Name of the End Station and code for End Terminal
- A serial number for the bike
- Subscriber type and zip code

**Question 1.** The dataset is in a file called `trip.csv`.  Load it into a table named `trips`:

In [ ]:
trips = Table.read_table('trip.csv') #SOLUTION
trips

**Question 2.** Create a new table `starts` that has two columns, `Start Station` that has the name of every station and `Number of Trips` that has the number of rides started at the corresponding station and a new table `Duration mean` that has two columns, `Start Station` that has the name of every station and `Average Duration` that has the average duration of rides from the corresponding station:

In [ ]:
starts = trips.group('Start Station').relabeled('count', 'Number of Trips') #SOLUTION
starts

In [ ]:
durations = trips.select('Start Station', 'Duration').group("Start Station", np.mean) #SOLUTION
durations

**Question 3.** Produce a bar graph with your new table with the stations listed in descending order:

In [ ]:
starts.sort('Number of Trips', descending=True).barh("Start Station", "Number of Trips") #SOLUTION

The file `station.csv` contains geographical information about each bike station, including latitude, longitude, and a "landmark" which is the name of the city where the station is located.

**Question 4.** Load `station.csv`.  Load it into a table named `stations`:

In [ ]:
stations = Table.read_table('station.csv') #SOLUTION
stations

**Question 5.** Draw a map of where the stations are located, using `Marker.map_table`. The function operates on a table, whose columns are (in order) latitude, longitude, and an optional identifier for each point:



In [ ]:
Marker.map_table(stations.select('lat', 'long', 'name')) #SOLUTION

The map is created using [OpenStreetMap](http://www.openstreetmap.org/#map=5/51.500/-0.100), which is an open online mapping system that you can use just as you would use Google Maps or any other online map. Zoom in to San Francisco to see how the stations are distributed. Click on a marker to see which station it is.

You can also represent points on a map by colored circles using `Circle.map_table`. 

**Question 6.** Make a table called `sf_map_data` with the `lat`, `long`, and `name` columns of just the San Francisco bike stations:

In [ ]:
sf_stations = stations.where('landmark', are.equal_to('San Francisco')) #SOLUTION
sf_map_data = sf.select('lat', 'long', 'name') #SOLUTION

Circle.map_table(sf_map_data, color='green', radius=200)

In this map we set `radius=200` which tells python how big to make the circles for our points. Wouldn't it be nice if we could combine information from our `starts` table and set the size of the circles to how many trips originated from that station? Let's do it.

**Question 6.** Define a function `find_trip_count` that takes a station name and returns the number of trips that started at that station and a function `find_average_duration` that takes a station name and returns the average duration:

In [ ]:
def find_trip_count(station_name):
    return starts.where("Start Station", are.equal_to(station_name)).column("Number of Trips").item(0) #SOLUTION

def find_average_duration(station_name):
    return durations.where("Start Station", are.equal_to(station_name)).column("Duration mean").item(0) #SOLUTION

Unfotunately some of the stations in the stations dataset are not present in the trips dataset. We must filter them out before applying find_trip_count to the remainder.

**Question 7.** Create a new table `count_by_station` that only has the rows for stations that are in the `trips` table. It should have the same columns as `stations` with an added column `Number of Trips` which contains the number of trips from that station and a column `Average Trip Duration` that contains the average trip duration of that station:



In [ ]:
stations_with_trip_data = stations.where("name", are.contained_in(starts.column("Start Station"))) #SOLUTION
count_by_station = stations_with_trip_data.with_columns( #SOLUTION
    "Number of Trips", #SOLUTION
    stations_with_trip_data.apply(find_trip_count, "name"), #SOLUTION
    "Average trip duration",
    count_by_station.apply(find_average_duration, "name"))
    
    
count_by_station

Finally, lets make our map! Below we have defined a function to help you with the colors.

In [ ]:
def duration_to_color(average_duration):
    """Converts an average trip duration to a string describing a color.
    
    Longer durations will be closer to bright red, and shorter durations
    will be closer to black.
    
    Args:
      average_duration (float): The average trip duration for one
        station.
    
    Returns:
      (string): A string describing a color based on the given average
        trip duration.  The string is in 6-digit hexidecimal format,
        which is a common way to describe colors."""
    max_duration_color = 255
    color_bits = 8
    rescaled_duration = min(max_duration_color, int(256 * average_duration / 5000))
    red_amount = 2**(2*color_bits) * rescaled_duration
    color = '#{:06X}'.format(red_amount)
    return color

**Question 8.** Create a table called `starts_map_data` with that has five columns: `lat`, `long`, `name`, `colors` and `area`. Use  `duration_to_color` and `find_trip_count` to make the values for the `colors` and `area` columns:

In [ ]:
starts_map_data = count_by_station.select('lat', 'long', 'name').with_columns( #SOLUTION
    'color', count_by_station.apply(duration_to_color, 'Average trip duration'), #SOLUTION
    'area', count_by_station.column('Number of Trips') * 4000, #SOLUTION
)
starts_map_data.show(3)
Circle.map_table(starts_map_data)

### Conclusions
It seems that the locations with long trip durations are mostly in Palo Alto and Redwood City, with one exception in San Jose.  These are the least urban bike stations on the map.  The data are therefore compatible with our hypothesis.

Until now, we have not proposed a causal mechanism for the association.  Here are a few that are plausible:

* Palo Alto and Redwood City are close to long bike routes in the hills to the southwest.  Perhaps people take long recreational biking trips through the hills.
* Perhaps Stanford students rent bicycles to get around campus for days at a time.
* Perhaps some people who live or work in the long suburban peninsula between San Francisco and San Jose commute for long distances by bicycle.

**Question for thought:** The `trips` dataset includes the date and time of day for the start and end of each trip.  How might we use this information to test some of the proposals above?

## 2. Concepts in Hypothesis Testing


High blood pressure is thought to cause or be associated with several health risks, so medication is sometimes prescribed to lower blood pressure for people with that condition.  There are two common measurements of blood pressure: the pressure when your heart beats and pushes blood through your arteries (systolic), and pressure in between heartbeats (diastolic).  We'll focus on systolic pressure for simplicity.  Pressures are measured in units of millimeters of mercury, which is abbreviated "mmHg".

Suppose we conduct an observational study to determine the efficacy of a medication, PressureLow, for reducing blood pressure.  Surveying a group of American adults, we find that the average systolic blood pressure of people taking PressureLow is *higher* than the average blood pressure of people not taking PressureLow.

### Question 1
Describe the confounding factor you think would be most important in such a study.

**SOLUTION:** People take PressureLow because their blood pressure is high to begin with.

Suppose we forge ahead with an hypothesis test.  We decide to test the following null hypothesis:

> *Null hypothesis:* The people in our study were randomly assigned to receive the medication, and the medication had no effect.

### Question 2
What is the appropriate alternative hypothesis, given that this is our null hypothesis?

**SOLUTION:** *Alternative:* They weren't randomly assigned, **or** the medication had an effect.

Some (synthetic) data are loaded in the next cell.

In [13]:
d = Table.read_table("pressurelow.csv")
d

### Question 3
What does each row in the table `d` represent?

**SOLUTION:** A single person, with their medication status and blood pressure.

You decide to use the average blood pressure of the people taking PressureLow as your test statistic.

In [11]:
def bp_test_statistic(input):
    return np.mean(input)

### Question 4
Use the test statistic function to compute the observed test statistic.

In [12]:
observed_test_statistic = bp_test_statistic(d.where("Medication", are.equal_to("Yes")).column("Systolic Pressure")) #SOLUTION
observed_test_statistic

### Question 5
**True** or **false** and **explain:** Calling the function `simulate_test_stat_under_null` (defined in the cell below) *with an argument of 1000* is a reasonable way to simulate a test statistic we could see if the null hypothesis were true.

In [14]:
def simulate_test_stat_under_null(sample_size):
    sample = d.sample(sample_size, with_replacement=False)
    pressures = sample.column("Systolic Pressure")
    return bp_test_statistic(pressures)

**SOLUTION:** True.  The function samples people's blood pressures at random from the whole observed population.  Under the null hypothesis, this is the same way the actual data were generated, since the medication made no difference.  This is what we did in class when testing a similar hypothesis about section grades.

The following histogram displays the distribution of test statistics from datasets simulated correctly under the null hypothesis.

![Simulated test statistics histogram](simulated_stats_hist.png)

### Question 6
Do you reject the null hypothesis, or not?

**SOLUTION:** Yes, we reject the null hypothesis.  The observed test statistic is 130.476, which is far outside the range of blood pressures that we would see if the null hypothesis were true, as displayed in the histogram.

### Question 7
Does this mean that PressureLow indeed caused higher blood pressure?  **Explain.**

**SOLUTION:** It does not.  Informally, our test has shown that the higher average blood pressure isn't "due to chance," but it could still be due to *confounding factors*.  Formally, our null hypothesis makes two claims: (1) people were assigned randomly, and (2) the drug had no effect.  If the first claim is false, then the second could still be true.  And we already knew the first claim was false before we ran our test, because we didn't randomize assignment of the drug.  So the test was essentially *useless*.

## 3. The Support for BCRA


If you've been following the news, you might know that Republicans in the US Congress are attempting to pass a bill about health care.  Various bills have been offered over the past few months.

Pollsters have found that many of these bills are unpopular among Americans.  For example, [this article](http://www.npr.org/2017/06/28/534612954/just-17-percent-of-americans-approve-of-republican-senate-health-care-bill) describes a pollster's findings about the popularity of the Better Care Reconciliation Act (BCRA).

When the pollsters tried to figure out the popularity of BCRA, they were trying to learn the *proportion of registered US voters* who would say that they approve of the bill.  They couldn't ask every registered voter, so instead they asked a random sample of 995.

In this exercise, we will simulate such random sampling to understand what happens.  How much random variability do we see in the proportion of sampled voters who approve of the bill?

We will imagine that we have actually asked all the registered voters about BCRA.  (Actually, the dataset only has 100,000 responses, because we found that the servers you're provided have a hard time handling all 200 million registered voters!)  Those data are loaded in the following cell:

In [41]:
response_codes = Table.read_table("bcra_population.csv")
response_codes.group_barh("Response Code")

We encoded the responses as 0, 1, 2, or 3 to save space.  0 refers to "Approve", 1 to "Disapprove", 2 to "Heard of it, just unsure", and 3 to "Have not heard enough about it to have an opinion".

### Question 1
Write code to produce a new table called `population`.  It should have a single column called `"Response"`, and it should have one row for each row in `response_codes`.  Instead of a response code like 0, it should have the corresponding string like `"Approve"`.

*Hint 1:* It should start like this:

|Response|
|-|
|Approve|
|Have not heard enough about it to have an opinion|
|Disapprove|

<p align="center">... (99997 rows omitted)</p>

*Hint 2:* Define a function and use `apply`.  Your function can reference the `responses` array we've given you.

In [ ]:
# This array is provided for your convenience.
responses = make_array(
    "Approve",
    "Disapprove",
    "Heard of it, just unsure",
    "Have not heard enough about it to have an opinion")

def number_to_response(response_number):
    """Converts a number 0 through 3 to a response string."""
    return responses.item(response_number)

population = Table().with_columns("Response", response_codes.apply(number_to_response, "Response Code"))
population

### Question 2
Implement the function called `proportions`, according to the docstring given below.

* `"Response"`: The response.
* `"Proportion"`: The proportion of people in `population` with that response.

In [ ]:
def proportions(tbl, col_name):
    """Computes the frequency distribution of column col_name in table tbl.
    
    Args:
      - tbl (Table): Any table.
      - col_name (str): The name of a column in tbl.
    
    Returns:
      Table: A table containing the frequency distribution of column col_name
        in table tbl.  It has two columns: col_name and "Proportion".  It has
        one row for each unique value in the col_name column."""
    counts = tbl.group(col_name) #SOLUTION
    return counts.drop("count").with_columns("Proportion", counts.column("count") / tbl.num_rows) #SOLUTION

support_proportions = proportions(population, "Response")
support_proportions.barh("Response", "Proportion")

### Question 3
Sample **50** people from `population` without replacement, producing a table named `small_sample`.  **Then,** draw a bar chart of the frequency distribution of their responses.

*Hint:* You can do this in one line of code.

In [ ]:
small_sample = population.sample(50, with_replacement=False)
proportions(small_sample, "Response").barh("Response", "Proportion")

### Question 4
Compare the distribution of responses in the sample and the distribution of responses in the population.  Are they the same?  If not, how do they differ?

**SOLUTION:** The answer will vary depending on the sample, since it's random!  In our case, the people who disapprove are somewhat overrepresented in the sample, and the people who approve are very underrepresented.

### Question 5
If you sampled again, would you see the same pattern, or a different pattern?

**SOLUTION:** We would probably see a different pattern.

### Question 6
Repeat questions 3 through 5, but with a sample of 995.  Compare the distribution of responses to the small sample and the population.

In [ ]:
# Use this cell to run any code you need.
proportions(population.sample(995, with_replacement=False), "Response").barh("Response", "Proportion") #SOLUTION

**SOLUTION:** The distribution is much closer to the population distribution this time, since the sample size is larger.  They are so similar that it's hard to distinguish them visually at all.  It might be a good idea to show the raw numbers in a table, or to make a bar chart of the differences between the proportions.

### Question 7
The following cell defines some code.  Read it, run it, and describe:

* what each function does,
* how many people were sampled *in total* when you ran the cell,
* what is displayed in the chart it produces, and
* how many numbers are in the dataset whose distribution is represented in the histogram.

*Note:* It may take about a minute to run the cell.

In [ ]:
def simulate_approval_proportion(sample_size):
    the_sample = population.sample(sample_size, with_replacement=False)
    return the_sample.where("Response", are.equal_to("Approve")).num_rows / the_sample.num_rows

def approval_distribution(sample_size, num_simulations):
    simulation_sizes = Table().with_column("Sample size", np.repeat(sample_size, num_simulations))
    results = simulation_sizes.apply(simulate_approval_proportion, "Sample size")
    simulation_sizes.with_column("Proportion approving", results).hist("Proportion approving", bins=np.arange(0, 1+.02, .02))

approval_distribution(50, 2000)

**SOLUTION:** `simulate_approval_proportion` simulates a single random sample from the population, using its argument as the sample size.  It returns the proportion of people in the sample who approve.  `approval_distribution` calls that function many times and draws a histogram of all the proportions produced.  Therefore the histogram is a histogram of the 2,000 proportions of approvers in 2,000 samples of 50 people each from our population.

### Question 8
Run the next cell and compare the results with the previous histogram.  How many people were sampled in total when you ran the cell?  Describe how the *law of large numbers* can help explain the difference in the two histograms.

In [ ]:
approval_distribution(995, 2000)

**SOLUTION:** The distribution is "sharper" - more of the sample proportions were close to the population proportion, .17.  The law of large numbers tells us to expect that: as the sample size increases, the sample proportion of approvers should tend to be closer to the population proportion of approvers.  Note that we are also using sampling to approximate the distribution of sample proportions; we are sampling 2,000 times rather than taking all possible samples.  That means that rerunning either cell will give you a slightly different histogram.  But the phenomenon here is due entirely to the sample size (50 versus 995), not the number of simulations.